In [ ]:
!pip install scikit-image

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from skimage.io import imread,imshow,imsave
from skimage.color import rgb2gray
from skimage.filters import gaussian

-------------------
# Contents MP1
- ### 1-D convolution (discrete) - [30 points]
- ### 2-D convolution (discrete) - [40 points]
- ### Edge detection in images using convolution filters - [30 points]
-----------------

# 1-D Convolution (Discrete)

    A convolution is an integral that expresses the amount of overlap of one function g as it is shifted over another function f. It therefore "blends" one function with another. For example, in synthesis imaging, the measured dirty map is a convolution of the "true" CLEAN map with the dirty beam.

[Tutorial](https://www.youtube.com/watch?v=yyTu0SXeW1M) 6:30
![Definition of Discrete Convolution 1-D](./resources/convolution_1d_definition.jpg)

By its definition (Eq 2), a convolution operation between two discrete finite arrays are to be done on an infinite range of index k ∈ Z from k = -inf to k = inf. 

As a convention, we will consider the array starts at k = 0, that is, the first element of the array A is A\[0\]. 

E.g.: Signal A: [2, -1, 6, 1, 7, -3, 2, 6]

A[0] = 2, A[1] = -1, A[2] = 6, ..., A[7] = 6

This conviniently goes along with the array indexing in python. But what to do for the undefined elements, such as A[-1] and A[8]? -1 and 8 are valid values for index k ∈ Z.

We will assume any undefined element to be 0 in the array. We will go into further detail about this choice in the next few cells.

### [Exercise] Manual 1-D Convolution Excercise - [5 points]

With the definition above, calculate the results of convolutions of signals A, B, and C with Kernel H.

Signal A: \[2, -4, 6, 1, 7, -3, 2, 6\]

Signal B: \[0, 1, 0, 1, 9, 3, -1\]

Signal C: \[0, 0, 0, 0, 1, 0, 0, 0, 0\]

Kernel H: \[1, 2, -1, 3\]

Hint: The result array might not start with index k == 0. Mark the starting index for each result. (See the Turotial Above)

In [ ]:
## Code

### Answers: 

(\* denotes convolution operator.)

A * H = ?

B * H = ?

C * H = ?


### [Exercise] Convolution vs Cross-Correlation - [5 points]

![Definition of Discrete Correlation Filtering](./resources/correlation_1d_definition.jpg)

[1-D Convolution/Correlation Visualization](https://www.youtube.com/watch?v=O9-HN-yzsFQ) 0:36

Look at the definition of cross-correlation (Eq 4) and the visualization of the convolution and the cross-correlation examples. Compare the two operations.

What do you notice? In your words, how do the two operations differ from each other?

### Answer: ?

### [Exercise] 1-D Discrete Convolution in Python - [20 points]

Many common math/science Python packages offer optimized implementation of convolution function. Below is an example of Numpy's convolve function

In [ ]:
X = np.array([0, 1, 2, 3, 4])
H = np.array([0, 0.5, 1])

# Read: https://numpy.org/doc/stable/reference/generated/numpy.convolve.html
Y = np.convolve(X, H, mode="full")
Y_reverse = np.convolve(H, X, mode="full")

print(Y)
print(Y_reverse)

Now lets implement the 1-D convolution function corresponding to numpy's "full" mode in the cell below. However, the only numpy functions you can use are:
```
np.array()
np.zeros()
np.ones()
```

**Do not change the function signature (function name, return type, and input parameters)**

In [ ]:
def conv1d(X, H):
    '''
    Returns the result Y = X * H, where * is the 1-D convolution operator. (following variable convention in Eq 2)

        Parameters:
                X (array): Input data/signal
                H (array): Convolution kernel/signal

        Returns:
                Y (array): Convolution result similar to "full" mode in np.convolve()
    '''
    return

Now check your result by comparing it with ```np.convolve```. Notice that this is not a comprehensive test. You are encouraged to make your own testcases.

In [ ]:
print("Correct (Numpy) Result:")
np_result = np.convolve(X, H, mode="full")
print(np_result)

print()

print("Your Answer:")
your_result = conv1d(X, H)
print(your_result)


print("\nYour result is {}".format("correct!" if np.array_equal(np_result, your_result) else "incorrect."))

-----------------

# 2-D Convolution (Discrete)

![2-D Discrete Convolution Definition](./resources/convolution_2d_definition.jpg)

### Definition

Watch this 2-D discrete convolution example. [2-D Discrete Convolution Example](https://www.youtube.com/watch?v=_iZ3Q7VXiGI) 

Notice two things:

1. The convolution kernel is already flipped. By definition, the result of ```A (*convolve) B ``` is exactly the same as ```A (*cross-correlate) B'```, where ```B'``` is ```B``` flipped. [Read a little more about this](https://cs.stackexchange.com/questions/11591/2d-convolution-flipping-the-kernel). Therefore what we really see in the video is the cross-correlation of the Input image and the flipped kernel. This will continue to be the theme in this tutorial and in many discussions about convolution you'll see online.
2. The presenter uses the "valid" padding mode, meaning he will not consider the undefined elements of the input (no padding around the input). This contrasts other padding modes "full" and "same". You can read more about these modes at https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html


### Padding

[Convolution Padding Explanation](https://www.youtube.com/watch?v=6v05kAtV1M0) 11:13

![Different Padding Modes](./resources/0rs9l.gif)

### [Exercise] 2-D Discrete Convolution in Python - [40 points]

Now go ahead and implement the 2-D convolution function in the cell below. However, the only numpy functions you can use are:
```
np.array()
np.flipud()
np.fliplr()
np.zeros()
np.ones()
np.sum()
```
Typically, in image processing tasks, we want to preserve size of input after the convolution operation. **Therefore, your code should implement the "same" mode convolution here. You can assume stride=1 and padding fillvalue=0. You will need to figure out how much 0-padding you need based on the difference in the image and kernel size.** You can review https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html for reference.

**Do not change the function signature (function name, return type, and input parameters)**

In [ ]:
def conv2d(image, kernel):
    '''
    Returns the result Y = image * kernel, where * is the 2-D convolution operator.

        Parameters:
                image (ndarray): 2-D input image/signal
                kernel (ndarray): 2-D convolution kernel/signal

        Returns:
                Y (ndarray): 2-D convolution result similar to "same" mode in scipy/numpy.
    '''
    return

Now check your result by comparing it with ```scipy.signal.convolve2d```.

Notice that this is not a comprehensive test. You are encouraged to make your own testcases.

In [ ]:
image = np.array([[1,2,3],[2,3,4],[4,4,4],[-1,0,1]]) # 4 x 3
kernel = np.array([[1,2],[2,3.7]]) # 2 x 2

print("Correct (Scipy) Result:")
from scipy.signal import convolve2d
np_result = convolve2d(image, kernel, mode="same", boundary='fill', fillvalue=0)
print(np_result) # 4 x 3 - same as input since mode="same"

print()

print("Your Answer:")
your_result = conv2d(image, kernel)
print(your_result)

print("\nYour result is {}".format("correct!" if np.array_equal(
    np_result, your_result) else "incorrect."))

------------------

# Edge Detection in Images with Convolution

First download these sample images

In [ ]:
!curl "https://i.insider.com/56e7f946dd0895595d8b4792?width=700&format=jpeg&auto=webp" --output go.jpg
!curl "https://upload.wikimedia.org/wikipedia/commons/e/ef/MRI_Head_Brain_Normal.jpg" --output brain.jpg

In [ ]:
go = imread('go.jpg')
plt.figure(figsize=(10,10))
plt.title("A game of GO")
plt.imshow(go)
plt.show()

Let's convert the colored image to gray-scale by combining 3 R, G, and B channels to one.

This can be conveniently done with the "skimage.color.rgb2gray" function.

In [ ]:
go_gray = rgb2gray(go)
plt.figure(figsize=(10,10))
plt.gray()
plt.title("A game of GO in gray-scale")
plt.imshow(go_gray)
plt.show()

### [Exercise] Let's define some helper functions used for the rest of the assignment - [5 points]

In [ ]:
def plot_student_vs_scipy(image, kernel, student_f, scipy_f):
    '''
    Wrapper function that computes convolution using both students and scipy version and plots both results side-by-side for comparison.

        Parameters:
                image (array): 2-D input image/signal
                kernel (array): 2-D convolution kernel/signal
                student_f (function): the student's own convolution function
                scipy_f (function): the scipy convolution function

        Returns:
                None
    '''
    return

# check the difference in execution time between your implementation and numpy/scipy versions (refer to Labs for timing-related code)

### [Exercise] Study the construction of the given filters/kernels - [5 points]

In [ ]:
prewitt_y = np.array([[-1., -1., -1.],
                      [0.,0.,0.],
                      [1.,1.,1.]])
prewitt_x = prewitt_y.T # transpose operator

sobel_y = np.array([[-1.,-2.,-1.],
                    [0.,0.,0.],
                    [1.,2.,1.]])
sobel_x = sobel_y.T # transpose operator

Describe the effect of the two filters.

Do you notice any difference?

Can you give an explanation?

**Your Answer:** 

You can use the wrapper function to visually check the effect of the filters

In [ ]:
student_f = conv2d
scipy_f = scipy.signal.convolve2d

plot_student_vs_scipy(go_gray, prewitt_x, student_f, scipy_f)
plot_student_vs_scipy(go_gray, prewitt_y, student_f, scipy_f)


In [ ]:
plot_student_vs_scipy(go_gray, sobel_x, student_f, scipy_f)
plot_student_vs_scipy(go_gray, sobel_y, student_f, scipy_f)


### [Exercise] Design a new filter/kernel and explain what its effect is - [10 points]

Can you make a new 3 by 3 edge detection filter similar to the ones above?

Hint: make sure the sum of all elements in your filter is 0, so that the result image has the same overall magnitude as the input.

In [ ]:
your_filter = np.array([[0., 0., 0.],
                        [0., 0., 0.],
                        [0., 0., 0.]])

In [ ]:
plot_student_vs_scipy(go_gray, your_filter, student_f, scipy_f)

## [Exercise] Combine the two orientations of the non-symmetrical filters by applying pythagorean addition - [5 points]

Pseudocode for combining filters f1 and f2 to get a new filter f3:
```
f3 = sqrt(f1^2 + f2^2)
```

In [ ]:
def combine_filter(f1, f2):
    '''
    Combines f1 and f2 using above pseudocode.

        Parameters:
                f1 (array): filter 1
                f2 (array): filter 2

        Returns:
                f3 (array): filter 3
    '''    
    return f3

In [ ]:
sobel_combined = combine_filter(sobel_y, sobel_x)
plot_student_vs_scipy(go_gray, sobel_combined, student_f, scipy_f)

In [ ]:
prewitt_combined = combine_filter(prewitt_y, prewitt_x)
plot_student_vs_scipy(go_gray, prewitt_combined, student_f, scipy_f)

### [Exercise] Apply 1) Sobel, 2) Prewitt, and 3) their respective combined filters to the brain image and plot the results - [5 points]

In [ ]:
brain = rgb2gray(imread("brain.jpg"))